In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from sklearn.cluster import spectral_clustering
from sklearn.metrics import v_measure_score
import snf
from snf import metrics
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

### Papermill parameters

snf_matrix_path <br />
phenotypes_covariates_path <br />
metabolomics_path <br />
mca_dims_path <br />
output_dir_plots

In [ ]:
# Load numpy array of fused network
fused_network = np.loadtxt(snf_matrix_path, delimiter = ",")

In [ ]:
# Spectral clustering

# determine optimal number of clusters (estimated via an eigengap approach)
best, second = snf.get_n_clusters(fused_network)

# Perform spectral clustering on the fused network
labels = spectral_clustering(fused_network, n_clusters=best)
labels_second = spectral_clustering(fused_network, n_clusters=second)

In [ ]:
# Functions


def sort_fused_network(fused_network, labels_array):
    # Make Pandas Dataframes
    df = pd.DataFrame(fused_network)
    df_labels = pd.DataFrame(labels_array)
    df_labels.columns = ["Label"]
    # sort label df
    df_labels = df_labels.sort_values(by=['Label'])
    # sort fused network df with sorted labels
    df = df.reindex(df_labels.index)
    df = df[df_labels.index]
    array = df.to_numpy()
    np.fill_diagonal(array, 0)
    return(array)

def make_heatmap(array, n_clusters):
    # Create heatmap
    heatmap = plt.imshow(array, cmap='hot', interpolation='nearest')

    # Set axis names, title etc.
    plt.xlabel('samples')
    plt.ylabel('samples')
    cbar = plt.colorbar(heatmap)
    cbar.ax.set_ylabel('sample correlations', loc="top")
    plt.suptitle('Fused network: sample correlations\nNumber of clusters: {:.2f}'.format(round(n_clusters)))
    plt.show()

    return(plt)
    
# Sort Fused networks according to labels found by spectral clustering
sorted_fused_network_best = sort_fused_network(fused_network, labels)
sorted_fused_network_second = sort_fused_network(fused_network, labels_second)

In [ ]:
make_heatmap(sorted_fused_network_best,  best)

In [ ]:
make_heatmap(sorted_fused_network_second, second)

In [ ]:
# Evaluation metrics

# Determine V-measure score (requiring true lables)
#v_score_1 = v_measure_score(labels, true_labels)
#v_score_2 = v_measure_score(labels_second, true_labels)

# Silhouette score
np.fill_diagonal(fused_network, 0)
sil = metrics.silhouette_score(fused_network, labels)
sil2 = metrics.silhouette_score(fused_network, labels_second)

# Affinity Z-score
zscore =  metrics.affinity_zscore(fused_network, labels)
zscore2 = metrics.affinity_zscore(fused_network, labels_second)

## Compare clusters to phenotype data

In [ ]:
# Find common IDs with -omics dataframes used for SNF


metabolomics = pd.read_csv(metabolomics_path, index_col=0) #metabolomics_values_mapped
metabolomics = metabolomics.dropna()

In [ ]:
# Phenotypes process out

phenotypes_data = pd.read_csv(phenotypes_covariates_path , index_col=0) #phenotype_covariates_data.csv

phenotypes_data = phenotypes_data[phenotypes_data.index.isin(metabolomics.index)]
phenotypes_data.shape

# Add cluster labels from SNF

phenotypes_data["fused_label"] = labels
phenotypes_data["fused_label_2nd"] = labels_second

for i in phenotypes_data.index:
    phenotypes_data.at[i, 'fused_label'] =  "SNF_"+ str(phenotypes_data.at[i, 'fused_label'])
    phenotypes_data.at[i, 'fused_label_2nd'] =  "SNF_"+ str(phenotypes_data.at[i, 'fused_label_2nd'])

In [ ]:
phenotypes_data.head()

In [ ]:
# Function to generate the mosaic plot

from statsmodels.graphics.mosaicplot import mosaic

def make_mosaic(df, col1, col2, out_dir):
    
    #Sort df on pheno value that is plotted against the clusters
    df = df.sort_values(by=[col1])
    
    
    #Adjust plot size to value counts
    number_of_pheno_values = len(df[col1].value_counts())
    number_of_clusters = len(df[col2].value_counts())
    
    if number_of_pheno_values < 4:
        number_of_pheno_values = 4
    
    fig, ax = plt.subplots(figsize=(number_of_pheno_values*2,number_of_clusters*1.5))
    
    # Generate plot
    mosaic(df, [col1, col2], ax=ax, axes_label=False)
    plt.xlabel(col1, fontsize=20)
    plt.ylabel(col2, fontsize=20)
    plt.savefig(out_dir)
    plt.show()
    plt.close()
    return(plt)

In [ ]:
make_mosaic(phenotypes_data, 'Vitamines', 'fused_label', str(output_dir_plots) + 'Vitamines_l1.png')

In [ ]:
make_mosaic(phenotypes_data, 'Vitamines', 'fused_label_2nd', str(output_dir_plots) + 'Vitamines_l2.png')

In [ ]:
make_mosaic(phenotypes_data, 'Sex', 'fused_label', str(output_dir_plots) + 'sex_l1.png') 

In [ ]:
make_mosaic(phenotypes_data, 'Sex', 'fused_label_2nd', str(output_dir_plots) + 'sex_l2.png')

In [ ]:
make_mosaic(phenotypes_data, 'Sick', 'fused_label', str(output_dir_plots) + 'Sick_l1.png')

In [ ]:
make_mosaic(phenotypes_data, 'Sick', 'fused_label_2nd', str(output_dir_plots) + 'Sick_l2.png')

In [ ]:
make_mosaic(phenotypes_data, 'Menstruation', 'fused_label', str(output_dir_plots) + 'Menstruation_l.png')

In [ ]:
make_mosaic(phenotypes_data, 'Menstruation', 'fused_label_2nd', str(output_dir_plots) + 'Menstruation_l2.png')

In [ ]:
make_mosaic(phenotypes_data, 'Vitamines', 'fused_label', str(output_dir_plots) + 'Vitamines_l.png')

In [ ]:
make_mosaic(phenotypes_data, 'Vitamines', 'fused_label_2nd', str(output_dir_plots) + 'Vitamines_l2.png')

## Compare clusters on MCA dimensions

In [ ]:
mca_coordinates = pd.read_csv(mca_dims_path, index_col=0)
mca_coordinates = mca_coordinates[mca_coordinates.index.isin(phenotypes_data.index)]
phenotypes_data = phenotypes_data[phenotypes_data.index.isin(mca_coordinates.index)]


# Add cluster labels from SNF

mca_coordinates["fused_label"] = phenotypes_data['fused_label']
mca_coordinates["fused_label_2nd"] = phenotypes_data['fused_label_2nd']

for i in mca_coordinates.index:
    mca_coordinates.at[i, 'fused_label'] =  str(mca_coordinates.at[i, 'fused_label'])
    mca_coordinates.at[i, 'fused_label_2nd'] =  str(mca_coordinates.at[i, 'fused_label_2nd'])

mca_coordinates.columns = mca_coordinates.columns.str.replace(' ', '_')

In [ ]:
mca_coordinates

In [ ]:
# Shapiro tests show that MCA dimensions are not normalliy distributed

for i in mca_coordinates.columns[0:-2]:
    print(i + '\n')
    print(stats.shapiro(mca_coordinates[i]))
    print('_____________________________________________________\n\n')

In [ ]:
group1 = mca_coordinates[mca_coordinates['fused_label'] == 'SNF_0']
group2 = mca_coordinates[mca_coordinates['fused_label'] == 'SNF_1']

In [ ]:
for i in mca_coordinates.columns[0:-2]:
    print(i + '\n')
    print(stats.mannwhitneyu(group1[i], group2[[i]]))
    print('_____________________________________________________\n\n')

In [ ]:
group_a = mca_coordinates[mca_coordinates['fused_label_2nd'] == 'SNF_0']
group_b = mca_coordinates[mca_coordinates['fused_label_2nd'] == 'SNF_1']
group_c = mca_coordinates[mca_coordinates['fused_label_2nd'] == 'SNF_2']
group_d = mca_coordinates[mca_coordinates['fused_label_2nd'] == 'SNF_3']

In [ ]:
def anova_table(df, numerical, categorical):
    formula = numerical + ' ~ C(' + categorical + ')'
    model = ols(formula, data = df).fit()
    anova_table = sm.stats.kruskal(model, typ=2)
    print(anova_table)
    return(anova_table)

In [ ]:
for i in mca_coordinates.columns[0:-2]:
    print(i + '\n')
    print(stats.kruskal(group_a[i], group_b[i], group_c[i], group_d[i]))
    print('_____________________________________________________\n\n')